## This script translate the instruction set by splitting

In [ ]:
!wget https://raw.githubusercontent.com/gururise/AlpacaDataCleaned/main/alpaca_data_cleaned.json

In [ ]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

In [ ]:
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
! pip install mosestokenizer subword-nmt
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install ./
! pip install xformers
%cd ..

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# add fairseq folder to python path
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"
# sanity check to see if fairseq is installed
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [ ]:
# download the indictrans model


# downloading the indic-en model
# !wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/indic2en.zip
# !unzip indic2en.zip

# downloading the en-indic model
!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/en2indic.zip
!unzip en2indic.zip

# # downloading the indic-indic model
# !wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/m2m.zip
# !unzip m2m.zip

%cd indicTrans

In [ ]:
from indicTrans.inference.engine import Model

indic2en_model = Model(expdir='../en-indic')

In [ ]:
ta_sents = ['Happy birthday! May this special day bring you joy, laughter, and all the blessings you deserve. Have a wonderful year ahead!',
            'During the last member meeting, create a list of 5 ideas to improve the club.',
            'Design an algorithm to find the maximum profit of a stock buy and sell?']


indic2en_model.batch_translate(ta_sents, 'en', 'or')

In [ ]:

ta_paragraph = """The U.S. economy more than doubled in size between 1989 and 2019, largely due to the rise of the information technology sector, improvements in worker productivity and consumer demand, and government investments. This growth was faster than in the prior two decades."""

indic2en_model.translate_paragraph(ta_paragraph, 'en', 'or')

In [ ]:
# Recommeding to store the data in mounted google drive
# !mkdir /content/drive/MyDrive/tranlation_work/translated_data /content/drive/MyDrive/tranlation_work/translated_data/data /content/drive/MyDrive/tranlation_work/translated_data/error 

In [ ]:
import os

translated_files = set(os.listdir('/content/drive/MyDrive/tranlation_work/translated_data/data'))

In [ ]:
#Translate in chunks (e.g. 1K/5K)
# import openai
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# gdrive_path = '/content/drive/MyDrive/OdiaGen/Data/translated_instruction_data/'

# Replace 'your_api_key' with your actual API key
# openai.api_key = 'YOUR_API_KEY_COMES_HERE'

def translate_text(value):
    if "\n" in value:
      replace_dn = value.replace("\n\n", "\n")
      split_lines = replace_dn.splitlines()
      odia_splits = indic2en_model.batch_translate(split_lines, 'en', 'or')
      response = "\n".join(odia_splits)
    else:
      response = indic2en_model.translate_paragraph(value, 'en', 'or')
    return response.strip()

def translate_item(item):
    translated_item = {}
    for key, value in item.items():
        if value:
            translated_value = translate_text(value)
            translated_item[key] = translated_value
            translated_item['english_' + key] = value
        else:
            translated_item[key] = ''
            translated_item['english_' + key] = ''
    return translated_item


def save_item(item, file_name):
  with open(file_name, 'w') as f:
    json.dump(item, f, ensure_ascii=False, indent=4)


def translate_save(item, i):
    if f"translated_{i}.json" in translated_files:
      return
    if os.path.isfile( f"/content/drive/MyDrive/tranlation_work/translated_data/data/translated_{i}.json" ):
      return
    try:
      translated_item = translate_item(item)
      save_item(translated_item, f"/content/drive/MyDrive/tranlation_work/translated_data/data/translated_{i}.json")
      print(f"translated_{i}.json", "<---- succes")
    except Exception as e:
      print(f"translated_{i}.json: {e}")
      with open(f"/content/drive/MyDrive/tranlation_work/translated_data/error/translated_{i}.json", 'a'):
          pass

# Maximum number of parallel requests
MAX_PARALLEL_REQUESTS = 50

# Assuming the input JSON is in a file named 'input.json'
with open('/content/alpaca_data_cleaned.json', 'r') as f:
    data = json.load(f)
    print(len(data))

    

start = 0
#end = 52000
# steps = 100
end = 10000
translated_data = []
data = data[start:end]


with ThreadPoolExecutor(max_workers=MAX_PARALLEL_REQUESTS) as executor:
    futures = {executor.submit(translate_save, item, i) for i, item in enumerate(data)}
    
    # for future in tqdm(as_completed(futures), total=len(futures), desc="Translating"):
    #     translated_data.append(future.result())

# Save the translated data to a new JSON file named 'translated_data.json'
#with open(f'translated_data_up_to_{start}_to_{end}.json', 'w') as f:
#    json.dump(translated_data, f, ensure_ascii=False, indent=4)
# with open(f'{gdrive_path}translated_data_up_to_{start}_to_{end}.json', 'w') as file:
#   json.dump(translated_data, file, ensure_ascii=False, indent=4)
# !cat '{gdrive_path}translated_data_up_to_{start}_to_{end}.json'

print(f"Translation complete. The translated data is saved in 'translated_data_up_to -> {start} _to_ {end}'")

In [ ]:
def merge_json_files(data_folder):
    merged_data = []
    for i in range(50):
        print(i)
        
        file_path = os.path.join(data_folder, f"translated_{i}.json")

        if not os.path.isfile( file_path ):
          continue
        
        with open(file_path, 'r', encoding="utf-8") as file:
            data = json.load(file)
            merged_data.append(data)
    return merged_data

def write_merged_json_file(output_file, merged_data):
    with open(output_file, 'w', encoding="utf-8") as file:
        json.dump(merged_data, file, indent=2, ensure_ascii=False)

data_folder = '/content/drive/MyDrive/tranlation_work/translated_data/data'
output_file = '/content/drive/MyDrive/tranlation_work/odia_alpaca_data.json'

merged_data = merge_json_files(data_folder)
write_merged_json_file(output_file, merged_data)

In [ ]:
#from google.colab import files
#files.download('translated_data_up_to_0_to_1.json')

In [ ]:
#!pwd